<a href="https://colab.research.google.com/github/mohitmeht/Research_codes/blob/master/Notebook_on_SSB_EIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Writing the data

The thickness of the space-charge layer: $\kappa^{-1}=\sqrt{\frac{k_{B}T\epsilon_{0}\epsilon_{gb}}{2N_{A}e^{2}c_{gb}}}$. The area specific resistance of the grain is $R_{\Omega}^g$. The site average net capacitance of the grain is $\overline{c}_g$

In [0]:
import scipy.constants as sciconst
import numpy as np
from numba import njit

Vt = sciconst.k*300/sciconst.e # V
cgb = 1000 # mol/cm3

@njit(parallel=True,fastmath=True)
def TheSiteDistribution(s=0,a=0.1):
    gam = 1-a
    num1 = 1
    den1 = np.pi
    brac1 = np.divide(num1,den1)
    num2 = np.sin(a*np.pi)
    den2 = np.exp(gam*s) + np.exp(-gam*s) - (2*np.cos(a*np.pi))
    brac2 = np.divide(num2,den2)
    Fs = np.multiply(brac1,brac2)
    return Fs
    
@njit(fastmath=True)
def Rg_ohm():  # ohm-cm2
    return 2.

@njit(fastmath=True)
def cg_avg():  # F/cm2
    return 0.2e-9 

@njit(parallel=True,fastmath=True)
def cg_s(s=0):  # F/cm2
    return np.multiply(cg_avg(),np.exp(s))

@njit(parallel=True,fastmath=True)
def tau_grain_s(s=0):  # F/cm2
    return np.multiply(Rg_ohm(),cg_s(s)) # Grain resistance and site dependent capacitance

@njit(parallel=True,fastmath=True)
def c_gb_H(epsi_gb_H,r_H):  # F/cm2
    epsi_0 = sciconst.epsilon_0;
    epsi   = np.multiply(epsi_0,epsi_gb_H)
    cgbH   = np.divide(epsi,r_H)
    return cgbH

### Constants used in the article

In [0]:
gamma = 0.5 # heterogeneous
R_g_c = 183.5 # ohm-cm2 (222, 267)
T = 298.15 # K
tau_g_0 = 0.05e-6 #s
Dgb = 1.3e-9 #cm2/s
tau_gb_H = 1.2e-6 # s
tau_gb_D = 0.1e-6 # s
A0 = 1.1 # cm2
gamma_gb = 0.6
gamma_g = 0.85
R_gb_c = 2400 #ohm-cm2
sigma_g = 0.76e-3 # S/cm
sigma_gb = 0.13e-3 # S/cm
c_g = cg_avg() # F/cm2

### The equations for a solid state electrolyte is given as
\begin{equation}
Z_{SSE}(\omega) = Z_{H}(\omega) + Z_{G}^{gb}(\omega) + R_{\Omega}
\end{equation}
where \begin{equation}
Z_{H} = \frac{z_{H}}{A_{0}}
\end{equation} 
, and 
\begin{equation}
z_{H} = (\bar{z}^{g} + z_{H}^{gb}).
\end{equation}

$\bar{z}^{g}$ is the average impedance of grain,
\begin{equation}
\bar{z}^{g}\left(\omega,\tau^{0}_{c},\tau^{g}_{c},\gamma_{g},\bar{c}_{g}\right)=\frac{\tau_{g}^{0}/\bar{c}_{g}}{1-\left[1+\tau_{g}^{0}/\tau_{c}^{g}+\left(j\omega\tau_{g}^{0}\right)^{\gamma_{g}}\left(1+\tau_{g}^{0}/\tau_{c}^{g}\right)^{1-\gamma_{g}}\right]^{-1}}
\end{equation}
where $\tau_{g}^{0}=R_{\Omega}^{g}\bar{c}_{g}$, $\tau_{c}^{g}=R_{c}^{g}\bar{c}_{g}$, and $z_{H}^{gb}$ is the impedance of the grain boundary, 

\begin{equation}
z_{H}^{gb}\left(\omega,\tau_{H}^{gb},\tau_{c}^{gb},\gamma_{gb},c_{H}^{gb}\right)=\frac{\tau_{H}^{gb}/c_{H}^{gb}}{1-\left[1+\tau_{H}^{gb}/\tau_{c}^{gb}+\left(j\omega\tau_{H}^{gb}\right)^{\gamma_{gb}}\left(1+\tau_{H}^{gb}/\tau_{c}^{gb}\right)^{\left(1-\gamma_{gb}\right)}\right]^{-1}}.
\end{equation}

The compact layer at the grain-boundary is thick enough so that lithium ion concentration reaches bulk concentration (semi-infinite),
\begin{equation}
Z_{G}^{gb}\left(\omega,\tau_{D}^{gb},\sigma_{gb},\kappa_{gb},A_{0}\right)=\frac{1}{A_{0}\sigma_{gb}\kappa_{gb}\sqrt{1 + j\omega\tau_{D}^{gb}}}
\end{equation}
using 
\begin{equation}
Z_{0} = \frac{D_{gb}}{\sigma_{gb}}
\end{equation} 
the above equation can be rewritten as 
\begin{equation}
Z_{G}^{gb}\left(\omega\right)=\frac{Z_{0}}{A_{0}\left(D_{gb}\right)^{\frac{1}{2}}\sqrt{\frac{1}{\tau_{D}^{gb}}+j\omega}}
\end{equation}

#### The parameters needed for calculating the impedance of the grain ($\bar{z}^{g}$) are $\tau_{0}^{g}$, $\bar{c}_{g}$, $\tau_{c}^{g}$, $\gamma^{g}$, and $\omega$.

In [0]:
@njit(parallel=True,fastmath=True)
def z_g(tau_g_0,c_g,tau_g_c,gamma_g,w):
    time_ratio_0_c = tau_g_0/tau_g_c
    num = tau_g_0/c_g
    den1 = 1
    den21 = 1
    den22 = time_ratio_0_c
    den23a = np.power(1j*w*tau_g_0,gamma_g)
    den23b = np.power(1+time_ratio_0_c,(1-gamma_g))
    den23 = den23a*den23b
    den2 = np.power(den21 + den22 + den23,-1)
    den = den1 - den2
    zg = np.divide(num,den)
    return zg

#### The parameters needed for calculating the impedance for lithium ions travelling through the compact layer ($z_{H}^{gb}$) are $\tau_{H}^{gb}$, $c_{H}^{gb}$, $\tau_{c}^{gb}$, $\gamma^{gb}$, and $\omega$.

In [0]:
@njit(parallel=True,fastmath=True)
def z_gb_H(tau_gb_H,c_gb_H,tau_gb_c,gamma_gb,w):
    time_ratio_h_c = tau_gb_H/tau_gb_c
    num = tau_gb_H/c_gb_H
    den1 = 1
    den21 = 1
    den22 = time_ratio_h_c
    den23a = np.power(1j*w*tau_gb_H,gamma_gb)
    den23b = np.power((1+time_ratio_h_c),(1-gamma_gb))
    den23 = den23a*den23b
    den2 = np.power(den21 + den22 + den23,-1)
    den = den1 - den2
    zgbH = np.divide(num,den)
    return zgbH

#### The parameters needed for calculating the impedance in the compact layer ($Z_{H}$) are $z_{H}^{gb}$, $\bar{z}^{g}$, and $A_{0}$.

In [0]:
@njit(parallel=True,fastmath=True)
def Z_H(zgbH,zg,A0):
    z_H = zg + zgbH
    ZH = z_H/A0
    return ZH

#### The parameters needed to model impedance from grain and grain boundary ($Z_{G}^{gb}\left(\omega\right)$) are $D_{gb}$, $\sigma_{gb}$, $A_0$, $\tau_{D}^{gb}$, and $\omega$

In [0]:
@njit(parallel=True,fastmath=True)
def Z_gb_G(D_gb,sigma_gb,A0,tau_gb_D,w):
    Z0 = np.divide(D_gb,sigma_gb)
    num = Z0
    den1 = np.multiply(A0,np.sqrt(D_gb))
    den2a = np.divide(1.,tau_gb_D)
    den2b = np.multiply(1.0j,w)
    den2ab = den2a + den2b
    den2 = np.sqrt(den2ab)
    den = np.multiply(den1,den2)
    ZgbG = np.divide(num,den)
    return ZgbG

#### The total impedance becomes $Z_{SSE}=Z_{H}\left(\omega\right)+Z_{G}^{gb}\left(\omega\right)+R_{\Omega}$

The authors assumed $R_{\Omega} = 0$

In [0]:
@njit(parallel=True,fastmath=True)
def Z_SSE(ZH,ZgbG,Romega=0):
    ZSSE = ZH + ZgbG + Romega
    return ZSSE

In [0]:
@njit(parallel=True,fastmath=True)
def Z_SSE_ref(A0,tau_g_0,c_g,tau_g_c,gamma_g,tau_gb_H,c_gb_H,tau_gb_c,gamma_gb,D_gb,sigma_gb,tau_gb_D,w):
    ZH = Z_H(A0,tau_g_0,c_g,tau_g_c,gamma_g,tau_gb_H,c_gb_H,tau_gb_c,gamma_gb,w)
    ZgbG = Z_gb_G(D_gb,sigma_gb,A0,tau_gb_D,w)
    Romega = 0
    ZSSE = ZH + ZgbG + Romega
    return ZSSE

#### The frequency ranges used to match experiments from 1Hz-1MHz

In [0]:
f = np.logspace(-3,6,100) # in Hertz

#### Converting linear frequency to angular frequency

In [0]:
w = 2*np.pi*f # in radians

### Reproducing Figure 5 from Rama Kant's paper. The values used to create the plot are
\begin{align}
R_{c}^{g} &= 183.5\textrm{ }\Omega\textrm{cm}^2\\
T &= 298.15\textrm{ K} \\
\tau^{0}_{g} &= 0.05 \textrm{ }\mu\textrm{s}\\
D_{gb} &= 1.3\times 10^{-9} \textrm{ cm}^2/\textrm{s}\\
\tau_{H}^{gb} &= 1.2 \textrm{ }\mu\textrm{s}\\
\tau_{D}^{gb} &=  0.1 \textrm{ }\mu\textrm{s}\\
A_{0} &= 1.1 \textrm{ cm}^2\\
\gamma_{gb} &=  0.6 \\ 
\gamma_{g} &= 0.85\\
R_{c}^{gb} &=2400\textrm{ }\Omega\textrm{cm}^2\\
\sigma_{g} &= 0.76\textrm{ mS/cm}\\
\sigma_{gb} &= 0.13\textrm{ mS/cm}
\end{align}

In [0]:
R_g_c     = 183.5 # ohm-cm2
T         = 298.15 # K
tau_g_0   = 0.05e-6 #s
D_gb      = 1.3e-9 # cm2/s
tau_gb_H  = 1.2e-6 #s
tau_gb_D  = 0.1e-6 #s
A0        = 1.1 # cm2
gamma_gb  = 0.6
gamma_g   = 0.85 
R_gb_c    = 2400 # ohm-cm2
sigma_g   = 0.76e-3 #S/cm
sigma_gb  = 0.13e-3 #S/cm

### The other parameters the authors might have used,  
\begin{align}
c_{H}^{gb} &= 1\textrm{ }\mu\textrm{F/cm}^2\\
\epsilon_{H}^{gb} &=  2\\
R_{\Omega}^{g} &=2\textrm{ }\Omega\textrm{cm}^2\\
\bar{c}_{g} &=0.2 \textrm{ nF/cm}^2\\
c_{\textrm{Li}}& = 1\textrm{ M}
\end{align}

In [0]:
c_gb_H    = 1e-6 #F/cm2
c_g       = 0.2e-9 #F/cm2
tau_gb_c  = R_gb_c*c_gb_H
tau_g_c   = R_g_c*c_g

## Computing all impedances

In [0]:
zg = z_g(tau_g_0,c_g,tau_g_c,gamma_g,w)

In [0]:
zgbH = z_gb_H(tau_gb_H,c_gb_H,tau_gb_c,gamma_gb,w)

In [0]:
ZH = Z_H(zgbH,zg,A0)

In [0]:
ZgbG = Z_gb_G(D_gb,sigma_gb,A0,tau_gb_D,w)

In [0]:
ZSSE = Z_SSE(ZH,ZgbG)

### Dataframe used to collect all the relavent data

In [0]:
import pandas as pd
df = pd.DataFrame()
df['f'] = f
df['zg'] = zg
df['zgbH'] = zgbH
df['ZH'] = ZH
df['ZgbG'] = ZgbG
df['ZSSE'] = ZSSE

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(np.real(ZSSE),-np.imag(ZSSE),'-')
realZSSE = np.real(ZSSE)
imagZSSE = -np.imag(ZSSE)

In [0]:
fig1 = plt.figure(num=1,figsize=[3,3],facecolor=None,edgecolor=None,dpi=150)
ax1 = fig1.add_subplot(111)
ax1.set_facecolor('None')
ax1.set_xlabel('$\omega$ [s$^{-1}$]')
ax1.set_ylabel('|Z($\omega$)|')  
ax1.semilogx(w,np.abs(ZSSE),color='tab:blue')
ax2 = ax1.twinx()
ax2.semilogx(w,-np.angle(ZSSE,deg=True),color='tab:red')
ax2.set_ylabel('$\Phi$ [degree]') 
plt.show()

In [0]:
zg = z_g(tau_g_0,c_g,tau_g_c,gamma_g,w)
plt.loglog(f,np.abs(zg));

### Testing the response of grain impedance from the model

In [0]:
plotval = zg
color = 'tab:red'
fig, ax1 = plt.subplots()
ax1.set_xlabel('Frequency [Hz]')
ax1.set_ylabel('log|Z($\omega$)|')  
ax1.plot(np.log10(w),np.log10(np.abs(plotval)))
ax1.set_ylim(1.4,3.6)
ax2 = ax1.twinx()
ax2.plot(np.log10(w),-np.angle(plotval,deg=True),color=color)
ax2.set_ylabel('$\Phi$ [degree]') 
plt.show()

In [0]:
plotted_tau_g_0 = tau_g_0
plotted_c_g = c_g
plotval = z_g(plotted_tau_g_0,plotted_c_g,tau_g_c,gamma_g,w)
fig1 = plt.figure(num=1,figsize=[3,3],facecolor=None,edgecolor=None,dpi=150)
ax1 = fig1.add_subplot(111)
ax1.set_facecolor('None')
ax1.plot(np.log10(w),np.log10(np.abs(plotval)),color='tab:cyan',alpha=0.7)
ax1.set_xlabel('log($\omega$) [Hz]')
ax1.set_ylabel('log|Z($\omega$)|')  
ax1.set_ylim(1.4,3.6)
ax1.set_yticks([1.5,2,2.5,3,3.5])
ax1.minorticks_on()
ax1.tick_params(axis='both',direction='in',which='both')
ax2 = ax1.twinx()
ax2.plot(np.log10(w),-np.angle(plotval,deg=True),color='tab:red')
ax2.set_ylim(-1,55);